# 1. Weight and Bias Login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_06"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

# 2. Login Huggingface

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 3. Dataset Load

In [3]:
from datasets import load_dataset

In [4]:
ko_instruction_01 = load_dataset("nlpai-lab/kullm-v2", cache_dir="/mnt/t7/.cache/huggingface/datasets", split="train")

In [5]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [6]:
ko_instruction_01

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [7]:
ko_instruction_01 = ko_instruction_01.train_test_split(test_size=0.92)

In [8]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 12210
    })
    test: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 140420
    })
})

In [9]:
ko_instruction_01["train"][100]

{'id': 'alpaca_{idx}',
 'instruction': '이 명령어에 입력이 제공되지 않는 이유를 설명하세요.',
 'input': '',
 'output': '이 명령은 사용자의 직접적인 질문이나 요청이 아니라 AI 어시스턴트인 저에게 정보를 제공하라는 프롬프트이기 때문에 입력이 제공되지 않은 것 같습니다. 대신 특정 사례가 아닌 일반적인 의미에서 입력이 제공되지 않는 상황에 대한 설명을 제공해 달라는 요청인 것으로 보입니다.'}

# 4. Loading the Model

In [10]:
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [12]:
# 4bit QLoRA 학습을 위한 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map, cache_dir=cache_model_dir, trust_remote_code=True)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_model_dir)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})


1

In [15]:
tokenizer.padding_side = "right"

In [16]:
model.resize_token_embeddings(len(tokenizer)) # pad_token이 추가되었으므로 embedding과 language modeling head를 resize

Embedding(46337, 4096)

# 5. LoRA Setup

In [17]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

In [18]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# 6. Formatting Dataset

In [19]:
def format_instruction(sample):
    system_prompt = f"### instruction: {sample['instruction']}"
    input = f"### input: {sample['input']}" if len(sample["input"]) > 0 else None
    output = f"### output: {sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [20]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/12210 [00:00<?, ? examples/s]

In [21]:
train_dataset["text"][100]

'### instruction: 이 명령어에 입력이 제공되지 않는 이유를 설명하세요.\n\n### output: 이 명령은 사용자의 직접적인 질문이나 요청이 아니라 AI 어시스턴트인 저에게 정보를 제공하라는 프롬프트이기 때문에 입력이 제공되지 않은 것 같습니다. 대신 특정 사례가 아닌 일반적인 의미에서 입력이 제공되지 않는 상황에 대한 설명을 제공해 달라는 요청인 것으로 보입니다.</s>'

# 7. Training Argument Setup

In [22]:
from transformers import TrainingArguments

In [23]:
output_dir = "./checkpoint/experi_09"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
report_to="wandb"
save_steps = 20
save_total_limit=5
num_train_epochs = 1
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [24]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    report_to = report_to,
    learning_rate=learning_rate,
    bf16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [25]:
from trl import SFTTrainer

max_seq_length = 1024

In [26]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/12210 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# 8. Training

In [28]:
trainer.train()

  0%|          | 0/6105 [00:00<?, ?it/s]

{'loss': 2.4093, 'grad_norm': 0.125, 'learning_rate': 1.0869565217391305e-05, 'epoch': 0.0}
{'loss': 2.5851, 'grad_norm': 0.1259765625, 'learning_rate': 2.173913043478261e-05, 'epoch': 0.0}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 2.8347, 'grad_norm': 0.2265625, 'learning_rate': 3.260869565217392e-05, 'epoch': 0.0}
{'loss': 3.2892, 'grad_norm': 0.53125, 'learning_rate': 4.347826086956522e-05, 'epoch': 0.01}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


...

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.7005, 'grad_norm': 0.59765625, 'learning_rate': 5.066711704104036e-07, 'epoch': 1.0}
{'loss': 1.6119, 'grad_norm': 1.09375, 'learning_rate': 1.6889039013680123e-07, 'epoch': 1.0}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'train_runtime': 7779.2766, 'train_samples_per_second': 1.57, 'train_steps_per_second': 0.785, 'train_loss': 1.757683207537677, 'epoch': 1.0}


TrainOutput(global_step=6105, training_loss=1.757683207537677, metrics={'train_runtime': 7779.2766, 'train_samples_per_second': 1.57, 'train_steps_per_second': 0.785, 'train_loss': 1.757683207537677, 'epoch': 1.0})

In [29]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("./results/experi_08", save_embedding_layers = True)

In [30]:
lora_config = LoraConfig.from_pretrained("./results/experi_09")
model = get_peft_model(model, lora_config)

ValueError: Can't find 'adapter_config.json' at './results/experi_09'

In [32]:
tokenizer.push_to_hub('aeolian83/llama_ko_sft_kullm_experi_06')

CommitInfo(commit_url='https://huggingface.co/aeolian83/llama_ko_sft_kullm_experi_06/commit/457621b3ada58baf6670b9069de6fb8982f02256', commit_message='Upload tokenizer', commit_description='', oid='457621b3ada58baf6670b9069de6fb8982f02256', pr_url=None, pr_revision=None, pr_num=None)